In [1]:
from flask import Flask
import openaq
from flask_sqlalchemy import SQLAlchemy

In [2]:
api = openaq.OpenAQ()

In [3]:
status, body = api.cities()

In [4]:
status

200

In [5]:
body

{'meta': {'name': 'openaq-api',
  'license': 'CC BY 4.0',
  'website': 'https://docs.openaq.org/',
  'page': 1,
  'limit': 100,
  'found': 2790,
  'pages': 28},
 'results': [{'city': 'Escaldes-Engordany',
   'country': 'AD',
   'locations': 2,
   'count': 78515},
  {'city': 'unused', 'country': 'AD', 'locations': 1, 'count': 5780},
  {'city': 'Dubai', 'country': 'AE', 'locations': 1, 'count': 17017},
  {'city': 'Abu Dhabi', 'country': 'AE', 'locations': 1, 'count': 24058},
  {'city': 'Buenos Aires', 'country': 'AR', 'locations': 4, 'count': 14976},
  {'city': 'Amt der K�rntner Landesregierung',
   'country': 'AT',
   'locations': 16,
   'count': 104663},
  {'city': 'Amt der Salzburger Landesregierung',
   'country': 'AT',
   'locations': 12,
   'count': 94197},
  {'city': 'Austria', 'country': 'AT', 'locations': 174, 'count': 121987},
  {'city': 'Amt der Nieder�sterreichischen Landesregierung',
   'country': 'AT',
   'locations': 39,
   'count': 322499},
  {'city': 'Umweltbundesamt', '

In [6]:
status, body = api.measurements(city='Los Angeles', parameter='pm25')

In [7]:
body['meta']

{'name': 'openaq-api',
 'license': 'CC BY 4.0',
 'website': 'https://docs.openaq.org/',
 'page': 1,
 'limit': 100,
 'found': 4099,
 'pages': 41}

In [8]:
body['results']

[{'location': 'Los Ángeles Oriente',
  'parameter': 'pm25',
  'date': {'utc': '2019-08-23T14:00:00.000Z',
   'local': '2019-08-23T11:00:00-03:00'},
  'value': 57,
  'unit': 'µg/m³',
  'coordinates': {'latitude': -37.463035064801, 'longitude': -72.324575155457},
  'country': 'CL',
  'city': 'Los Angeles'},
 {'location': '21 de mayo',
  'parameter': 'pm25',
  'date': {'utc': '2019-08-23T14:00:00.000Z',
   'local': '2019-08-23T11:00:00-03:00'},
  'value': 112,
  'unit': 'µg/m³',
  'coordinates': {'latitude': -37.471182288689, 'longitude': -72.36146284977},
  'country': 'CL',
  'city': 'Los Angeles'},
 {'location': 'Los Ángeles Oriente',
  'parameter': 'pm25',
  'date': {'utc': '2019-08-23T13:00:00.000Z',
   'local': '2019-08-23T10:00:00-03:00'},
  'value': 35,
  'unit': 'µg/m³',
  'coordinates': {'latitude': -37.463035064801, 'longitude': -72.324575155457},
  'country': 'CL',
  'city': 'Los Angeles'},
 {'location': '21 de mayo',
  'parameter': 'pm25',
  'date': {'utc': '2019-08-23T13:00:0

In [9]:
samples = [(entry['date']['utc'], entry['value']) for entry in body['results']]

# 3

In [10]:
# Instantiate app with Flask
APP = Flask(__name__)


# SQLAlchemy stuff
APP.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite3'
DB = SQLAlchemy(APP)


class Record(DB.Model):
    id = DB.Column(DB.Integer, primary_key=True)
    datetime = DB.Column(DB.String(25))
    value = DB.Column(DB.Float, nullable=False)

    def __repr__(self):
        return 'Time:' + str(self.datetime) + ', Value:' + str(self.value)

/home/eric/anaconda3/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [11]:
for sample in samples:
    measure = Record(datetime = sample[0], value = sample[1])
    DB.session.add(measure)

In [12]:
DB.session.commit()

In [25]:
Record.query.filter(Record.value >= 10.0)